## Get Fortune 500 data

We have manually downloaded the "loaded" html from https://www.50pros.com/fortune500, then, we are going to download the financial information of those companies that are publicly traded. 

Since we are unsure about their specific exchange we are first going to search the tickers in the Nasdaq and if some of the companies are left we are going to search the NYSE

In [1]:
import pandas as pd
from bs4 import BeautifulSoup

# Step 1: Read the HTML from the file
with open('fortune_500.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Step 2: Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Step 3: Find the table with id 'example'
table = soup.find('table', id='example')

# Step 4: Convert the HTML table to a pandas DataFrame
# Extract table headers
headers = []
for th in table.find_all('th'):
    headers.append(th.text.strip())

# Extract table rows
rows = []
for tr in table.find_all('tr')[1:]:  # skip the header row
    cells = []
    for td in tr.find_all('td'):
        cells.append(td.text.strip())
    rows.append(cells)

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

In [2]:
df

,Rank,Company,Industry,City,State,Zip,Website,Employees,"Revenue\n(in millions, USD)","Valuation\n(in millions, USD)","Profits\n(in millions, USD)",Profits\n(% of Sales),Ticker,CEO
0,1,Walmart,General Merchandisers,Bentonville,Arkansas,72716,www.walmart.com,"2,100,000","$611,289","$397,475","$11,680",1.9,WMT,Douglas McMillon
1,2,Amazon.com,Internet Services and Retailing,Seattle,Washington,98109,www.amazon.com,"1,541,000","$513,983","$1,058,440","-$2,722",-0.5,AMZN,Andrew R. Jassy
2,3,Exxon Mobil,Petroleum Refining,Irving,Texas,75039,www.exxonmobil.com,"62,000","$413,680","$446,424","$55,740",13.5,XOM,Darren W. Woods
3,4,Apple,"Computers, Office Equipment",Cupertino,California,95014,www.apple.com,"164,000","$394,328","$2,609,039","$99,803",25.3,AAPL,Timothy D. Cook
4,5,UnitedHealth Group,Health Care: Insurance and Managed Care,Minnetonka,Minnesota,55343,www.unitedhealthgroup.com,"400,000","$324,162","$440,854","$20,120",6.2,UNH,Andrew P. Witty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,KKR,Securities,New York,New York,10001,www.kkr.com,"4,150","$7,273","$45,225",-$841,-11.6,KKR,Joseph Y. Bae
496,497,Equinix,Real Estate,Redwood City,California,94065,www.equinix.com,"12,097","$7,263","$66,873",$704,9.7,EQIX,Charles Meyers
497,498,Sonoco Products,"Packaging, Containers",Hartsville,South Carolina,29550,www.sonoco.com,"22,000","$7,251","$5,975",$466,6.4,SON,Howard Coker
498,499,ServiceNow,Computer Software,Santa Clara,California,95054,www.servicenow.com,"20,433","$7,245","$94,338",$325,4.5,NOW,Bill Mcdermott


In [3]:
df["Ticker"].nunique()

477

We can see that 477/500 companies are publicly traded

Now that we have the companies and their tickers, we can search the API to see in which exchnages they are located, then, we can finally download the associated data

In [4]:
import enum
import requests
import yaml
import os
import pandas as pd

def get_api_key_from_yaml(yaml_file_path: str) -> str:
  """
  Reads the API key from a YAML file located at the specified path.
  """

  # Check if the YAML file exists
  if not os.path.exists(yaml_file_path):
    raise FileNotFoundError(f"YAML file not found: {yaml_file_path}")

  # Open and parse the YAML file
  with open(yaml_file_path, 'r') as f:
    try:
      data = yaml.safe_load(f)  # Use safe_load for security
    except yaml.YAMLError as e:
      raise ValueError(f"Error parsing YAML file: {e}")

  # Validate and return the API key
  api_key = data.get("api_key")
  if not api_key:
    raise KeyError(f"Missing 'api_key' key in YAML file: {yaml_file_path}")

  return api_key

In [5]:
api_path="/Users/fernando/Documents/GitHub/download-financial-data/api.yaml"
api_key = get_api_key_from_yaml(api_path)

## Identify stock exchange for each stock ticker

Note: This may not be necessary because we can do queries using the stock ticker

In [6]:
def stock_ticker_in_exchange(
        stock_ticker: str,
        stock_exchange: str,
        api_key: str,
        api_url: str = "https://financialmodelingprep.com/api/v3",
        result_limit:int = 10
) -> bool:
    """
    Confirms whether a stock ticker can be found in a stock exchange
    """
    # Construct the API query URL
    endpoint = "search-ticker"
    url = f"{api_url}/{endpoint}/?apikey={api_key}&query={stock_ticker}&exchange={stock_exchange}&limit={result_limit}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for non-200 status codes
    
        # Handle potential empty response (no data for the request)
        if not response.content:
          print(f"No data found for {stock_ticker} in {stock_exchange}")
          return False
    
        return len(response.json()) > 0
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return False

In [7]:
nasdaq_stock_exchange = "NASDAQ"
nyse_stock_exchange = "NYSE"

stock_ticker_in_exchange(
    stock_ticker="AAPL",
    stock_exchange=nasdaq_stock_exchange,
    api_key=api_key,
)

True

In [8]:
# from tqdm import tqdm
# 
# nasdaq_stock_exchange = "NASDAQ"
# nyse_stock_exchange = "NYSE"
# 
# for stock_ticker in tqdm(df["Ticker"]):
#     df.loc[df["Ticker"] == stock_ticker, nasdaq_stock_exchange] = stock_ticker_in_exchange(
#         stock_ticker=stock_ticker,
#         stock_exchange=nasdaq_stock_exchange,
#         api_key=api_key,
#     )
#     df.loc[df["Ticker"] == stock_ticker, nyse_stock_exchange] = stock_ticker_in_exchange(
#         stock_ticker=stock_ticker,
#         stock_exchange=nyse_stock_exchange,
#         api_key=api_key,
#     )

## Get Profile from ticker

From the profile we can also get other information such as industry, market cap, CEO and so on...

Note: We can also get the stock exchange if necessary

In [9]:
def get_profile(
        stock_ticker: str,
        api_key: str,
        api_url: str = "https://financialmodelingprep.com/api/v3",
):
    # Construct the API query URL
    endpoint = "profile"
    url = f"{api_url}/{endpoint}/{stock_ticker}?apikey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for non-200 status codes
    
        # Handle potential empty response (no data for the request)
        if not response.content:
            print(f"No data found for {stock_ticker}")
            return None
        return response.json()
    
    except requests.exceptions.RequestException as e:
          print(f"Error fetching data: {e}")
          return None

In [10]:
import pandas as pd
from tqdm import tqdm

profile_data = []
for stock_ticker in tqdm(df["Ticker"]):
    profile = get_profile(
        stock_ticker=stock_ticker,
        api_key=api_key,
    )
    profile_data.extend(profile)

100%|██████████| 500/500 [04:11<00:00,  1.99it/s]


In [11]:
df_profile = pd.DataFrame(profile_data)
df_profile

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,WMT,65.82,0.494,16222701,530495377800,0.83,49.503334-65.9,0.06,Walmart Inc.,USD,...,72716,22.35537,43.464628,https://financialmodelingprep.com/image-stock/...,1972-08-25,False,False,True,False,False
1,AMZN,178.34,1.155,39248707,1855913044000,0.00,118.35-191.7,1.90,"Amazon.com, Inc.",USD,...,98109-5210,128.74266,49.597345,https://financialmodelingprep.com/image-stock/...,1997-05-15,False,False,True,False,False
2,XOM,114.45,0.947,18073019,513414688500,3.80,95.77-123.75,-2.81,Exxon Mobil Corporation,USD,...,75039-2298,-40.58104,155.031036,https://financialmodelingprep.com/image-stock/...,1978-01-13,False,False,True,False,False
3,AAPL,194.03,1.264,62064611,2975275423000,1.00,164.08-199.62,1.78,Apple Inc.,USD,...,95014,57.87197,136.158029,https://financialmodelingprep.com/image-stock/...,1980-12-12,False,False,True,False,False
4,UNH,497.44,0.561,4223915,457836314400,7.52,436.38-554.7,2.07,UnitedHealth Group Incorporated,USD,...,55343,-403.31167,900.751672,https://financialmodelingprep.com/image-stock/...,1984-10-17,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,KKR,103.17,1.626,3413836,91553264340,0.70,52.95-108.0,0.33,KKR & Co. Inc.,USD,...,10001,13.76070,69.260700,https://financialmodelingprep.com/image-stock/...,2010-07-15,False,False,True,False,False
470,EQIX,751.46,0.618,731915,71317837322,17.04,677.8-914.93,-11.52,"Equinix, Inc.",USD,...,94065,802.82991,-51.369911,https://financialmodelingprep.com/image-stock/...,2000-08-11,False,False,True,False,False
471,SON,61.44,0.678,486669,6036836352,2.08,49.98-63.74,0.07,Sonoco Products Company,USD,...,29550,-20.81443,82.254426,https://financialmodelingprep.com/image-stock/...,1980-03-17,False,False,True,False,False
472,NOW,660.11,0.976,1293676,135322550000,0.00,526.11-815.32,3.18,"ServiceNow, Inc.",USD,...,95054,-18.97305,679.083049,https://financialmodelingprep.com/image-stock/...,2012-06-29,False,False,True,False,False


In [12]:
df_profile["symbol"].nunique()

474

In [13]:
df_profile["exchangeShortName"].value_counts()

exchangeShortName
NYSE      357
NASDAQ    114
PNK         2
AMEX        1
Name: count, dtype: int64

There are 2 companies from OTC exchange (PNK) but they seem to be fine

In [14]:
df_profile[(df_profile["exchangeShortName"]== "PNK")]

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
26,FNMA,1.54,1.374,4004119,1783458600,0.0,0.43-1.99,-0.010,Federal National Mortgage Association,USD,...,20005,NaN,0.00000,https://financialmodelingprep.com/image-stock/...,1977-01-03,False,False,True,False,False
42,FMCC,1.45,1.458,2087641,942587000,0.0,0.4-1.75,-0.015,Federal Home Loan Mortgage Corporation,USD,...,22102-3110,NaN,1.02345,https://financialmodelingprep.com/image-stock/...,1988-12-02,False,False,True,False,False


There are only two companies that are publicly traded and are not present in `df_profile`:
* Sillicon Valley Bank
* Berkshire Hathaway

The first one has been acquired by another bank, the second one, the ticker is wrong. The correct ticker is `BRK.A` but the API contains it as `BRK-A`

In [15]:
df[~df["Ticker"].isin(df_profile["symbol"])]

,Rank,Company,Industry,City,State,Zip,Website,Employees,"Revenue\n(in millions, USD)","Valuation\n(in millions, USD)","Profits\n(in millions, USD)",Profits\n(% of Sales),Ticker,CEO
6,7,Berkshire Hathaway,Insurance: Property and Casualty (Stock),Omaha,Nebraska,68131,www.berkshirehathaway.com,"383,000","$302,089","$675,657","-$22,819",-7.6,BRKA,Warren E. Buffett
43,44,State Farm Insurance,Insurance: Property and Casualty (Mutual),Bloomington,Illinois,61710,www.statefarm.com,"60,519","$89,328",,"-$6,654",-7.4,,Michael L. Tipsord
70,71,New York Life Insurance,"Insurance: Life, Health (Mutual)",New York,New York,10010,www.newyorklife.com,"15,050","$58,445",,"-$1,127",-1.9,,Craig Desanto
77,78,Publix Super Markets,Food and Drug Stores,Lakeland,Florida,33811,www.publix.com,"242,000","$54,942",,"$2,918",5.3,,Todd Jones
82,83,Nationwide,Insurance: Property and Casualty (Mutual),Columbus,Ohio,43215,www.nationwide.com,"24,791","$51,450",,$988,1.9,,Kirt A. Walker
85,86,Liberty Mutual Insurance Group,Insurance: Property and Casualty (Stock),Boston,Massachusetts,02116,www.libertymutual.com,"50,000","$49,956",,$414,0.8,,Timothy Sweeney
89,90,CHS,Food Production,Inver Grove Heights,Minnesota,55077,www.chsinc.com,"10,014","$47,792",,"$1,679",3.5,,Jay D. Debertin
102,103,TIAA,"Insurance: Life, Health (Mutual)",New York,New York,10017,www.tiaa.org,"16,070","$40,911",,$494,1.2,,
103,104,Massachusetts Mutual Life Insurance,"Insurance: Life, Health (Mutual)",Springfield,Massachusetts,01111,www.massmutual.com,"10,662","$40,281",,"$1,485",3.7,,Roger W. Crandall
110,111,Northwestern Mutual,"Insurance: Life, Health (Mutual)",Milwaukee,Wisconsin,53202,www.northwesternmutual.com,"8,323","$36,921",,$912,2.5,,John Schlifske


We can manually add Berkshire to the dataframe

In [16]:
profile_data.extend(get_profile("BRK-A", api_key=api_key))

df_profile = pd.DataFrame(profile_data)

## Get financial data

Now that we have the profiles of each company, we can get their financial statements

IMPORTANT: I had some issues when using the CIK, it seems better to use the ticker. With the CIK I got 681 tickers

In [17]:
class StatementType(enum.Enum):
  INCOME_STATEMENT = "income-statement"
  CASH_FLOW_STATEMENT = "cash-flow-statement"
  BALANCE_SHEET_STATEMENT = "balance-sheet-statement-as-reported"

def get_financial_statement_data(
        cik: str,
        api_key: str,
        statement_type: StatementType,
        api_url: str = "https://financialmodelingprep.com/api/v3/",
        period:str = "annual",
        period_limit:int = 5,
) -> dict[str, any] | None:
  """
  Fetches financial statement data for a given stock using the Financial Modeling Prep API.
  """

  # Validate the statement_type
  if not isinstance(statement_type, StatementType):
    raise ValueError(f"Invalid statement_type: {statement_type}. Must be a member of StatementType.")

  # Construct the API endpoint based on statement_type
  endpoint = statement_type.value

  # Construct the API query URL
  url = f"{api_url}{endpoint}/{cik}?apikey={api_key}&period={period}&limit={period_limit}"

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    # Handle potential empty response (no data for the request)
    if not response.content:
      print(f"No data found for {cik} for {statement_type.value}")
      return None

    return response.json()

  except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    return None


In [18]:
from tqdm import tqdm

financial_data_2 = []
number_of_years = 5

for cik in tqdm(df_profile["symbol"]):
  data = get_financial_statement_data(
      cik=cik,
      api_key=api_key,
      statement_type=StatementType.INCOME_STATEMENT, 
      period_limit=number_of_years,
  )
  financial_data_2.extend(data)

100%|██████████| 475/475 [03:32<00:00,  2.23it/s]


## Format final data

We need to organize the financial data by years. However, we cannot directly use the calendar year or date for grouping because some companies consider 2023 as 2024. Therefore, it's better to take the latest year and label it as 2023, the second latest as 2022, and so on.

With this in mind, we create a simple function called `assign_years()`

In [34]:
def assign_years(group, initial_year, total_years):
    years = [initial_year - i for i in range(total_years)]
    group['year'] = years
    return group

df_financial = pd.DataFrame(financial_data_2)
df_financial['date'] = pd.to_datetime(df_financial['date'])
df_financial = df_financial.sort_values(by=['symbol', 'date'], ascending=[True, False])

# Apply the function to each group
initial_year = 2023
number_of_years = 5
df_financial = df_financial.groupby('symbol').apply(assign_years, initial_year, number_of_years).reset_index(drop=True)

In [44]:
df_financial["year"].value_counts()

year
2023    475
2022    475
2021    475
2020    475
2019    475
Name: count, dtype: int64

In [45]:
df_finances = df_financial.groupby(['year', 'symbol'])[['revenue', "grossProfit", "operatingIncome", "netIncome"]].sum().reset_index()

df_merged = df_finances.merge(df_profile, on='symbol', how='left')

In [46]:
df_merged["symbol"].nunique()

475

In [47]:
df_final_renamed = df_merged.rename(columns={
  "grossProfit": "gross_profit",
  "operatingIncome": "operating_income",
  "netIncome": "net_income",
  "year": "calendar_year",
    "companyName": "name",
    "industry": "subSector"
})

In [48]:
df_final_renamed

,calendar_year,symbol,revenue,gross_profit,operating_income,net_income,price,beta,volAvg,mktCap,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,2019,AA,10495000000,1202000000,7.140000e+08,-853000000,44.01,2.449,6325826,7902435600,...,15212-5858,42.308160,1.701842,https://financialmodelingprep.com/image-stock/...,2016-11-01,False,False,True,False,False
1,2019,AAL,45768000000,10389000000,1.033000e+09,1686000000,11.53,1.580,32959903,7564717700,...,76155,2.015820,14.414200,https://financialmodelingprep.com/image-stock/...,2005-09-27,False,False,True,False,False
2,2019,AAP,9709003000,4254746000,6.771800e+08,486896000,67.42,1.221,1860538,4019735466,...,27604,-21.106950,88.526949,https://financialmodelingprep.com/image-stock/...,2001-11-29,False,False,True,False,False
3,2019,AAPL,260174000000,98392000000,6.393000e+10,55256000000,194.03,1.264,62064611,2975275423000,...,95014,57.871970,136.158029,https://financialmodelingprep.com/image-stock/...,1980-12-12,False,False,True,False,False
4,2019,ABBV,33266000000,25827000000,1.298300e+10,7882000000,160.19,0.593,5779576,282874715300,...,60064-6400,-901.282110,1063.372112,https://financialmodelingprep.com/image-stock/...,2013-01-02,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,2023,XEL,14206000000,3245000000,2.588000e+09,1771000000,55.28,0.366,5098207,30715723920,...,55401,0.564392,68.994400,https://financialmodelingprep.com/image-stock/...,2001-03-13,False,False,True,False,False
2371,2023,XOM,333711000000,82326000000,4.339600e+10,36010000000,114.45,0.947,18073019,513414688500,...,75039-2298,-40.581040,155.031036,https://financialmodelingprep.com/image-stock/...,1978-01-13,False,False,True,False,False
2372,2023,XPO,7744000000,770000000,4.380000e+08,189000000,102.37,2.022,1392826,11908804470,...,06831,134.037290,-31.667294,https://financialmodelingprep.com/image-stock/...,2003-10-07,False,False,True,False,False
2373,2023,YUMC,10978000000,1883000000,1.247000e+09,827000000,35.63,0.456,2927693,13892208260,...,200030,-65.974800,101.604802,https://financialmodelingprep.com/image-stock/...,2016-10-17,False,False,True,False,False


In [49]:
df_final_renamed[['calendar_year', 'symbol', 'name', 'sector', 'subSector', 'revenue', 'gross_profit',
       'operating_income', 'net_income', 'cik',]].to_csv("fortune_475_companies.csv", index=False)

In [50]:
df_final_renamed[['calendar_year', 'symbol', 'name', 'sector', 'subSector', 'revenue', 'gross_profit',
       'operating_income', 'net_income', 'cik',]]

,calendar_year,symbol,name,sector,subSector,revenue,gross_profit,operating_income,net_income,cik
0,2019,AA,Alcoa Corporation,Basic Materials,Aluminum,10495000000,1202000000,7.140000e+08,-853000000,0001675149
1,2019,AAL,American Airlines Group Inc.,Industrials,"Airlines, Airports & Air Services",45768000000,10389000000,1.033000e+09,1686000000,0000006201
2,2019,AAP,"Advance Auto Parts, Inc.",Consumer Cyclical,Specialty Retail,9709003000,4254746000,6.771800e+08,486896000,0001158449
3,2019,AAPL,Apple Inc.,Technology,Consumer Electronics,260174000000,98392000000,6.393000e+10,55256000000,0000320193
4,2019,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General,33266000000,25827000000,1.298300e+10,7882000000,0001551152
...,...,...,...,...,...,...,...,...,...,...
2370,2023,XEL,Xcel Energy Inc.,Utilities,Regulated Electric,14206000000,3245000000,2.588000e+09,1771000000,0000072903
2371,2023,XOM,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,333711000000,82326000000,4.339600e+10,36010000000,0000034088
2372,2023,XPO,"XPO Logistics, Inc.",Industrials,Integrated Freight & Logistics,7744000000,770000000,4.380000e+08,189000000,0001166003
2373,2023,YUMC,"Yum China Holdings, Inc.",Consumer Cyclical,Restaurants,10978000000,1883000000,1.247000e+09,827000000,0001673358


In [52]:
df_final_renamed[df_final_renamed["symbol"] == "JNJ"]

,calendar_year,symbol,revenue,gross_profit,operating_income,net_income,price,beta,volAvg,mktCap,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
242,2019,JNJ,82059000000,54503000000,2.100900e+10,15119000000,147.74,0.547,7680042,355562903200,...,08933,0.77045,146.969548,https://financialmodelingprep.com/image-stock/...,1943-01-02,False,False,True,False,False
717,2020,JNJ,82584000000,54157000000,1.982400e+10,14714000000,147.74,0.547,7680042,355562903200,...,08933,0.77045,146.969548,https://financialmodelingprep.com/image-stock/...,1943-01-02,False,False,True,False,False
1192,2021,JNJ,93775000000,63920000000,2.441700e+10,20878000000,147.74,0.547,7680042,355562903200,...,08933,0.77045,146.969548,https://financialmodelingprep.com/image-stock/...,1943-01-02,False,False,True,False,False
1667,2022,JNJ,94943000000,63854000000,2.391700e+10,17941000000,147.74,0.547,7680042,355562903200,...,08933,0.77045,146.969548,https://financialmodelingprep.com/image-stock/...,1943-01-02,False,False,True,False,False
2142,2023,JNJ,85152000000,58413000000,2.185300e+10,35153000000,147.74,0.547,7680042,355562903200,...,08933,0.77045,146.969548,https://financialmodelingprep.com/image-stock/...,1943-01-02,False,False,True,False,False


In [53]:
df_final_renamed[df_final_renamed["symbol"] == "APO"]

,calendar_year,symbol,revenue,gross_profit,operating_income,net_income,price,beta,volAvg,mktCap,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
41,2019,APO,2931849000,1670762000,1.312395e+09,1536843000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
516,2020,APO,2354019000,1265321000,1.412668e+09,466802000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
991,2021,APO,5951598000,2458218000,1.805842e+09,4267895000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
1466,2022,APO,3551356000,1608167000,1.158960e+08,1994320000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
1941,2023,APO,33213000000,32297000000,8.317000e+09,5047000000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False


We have observed an issue with Apollo, let's fix it manually

In [63]:
df_final_renamed = df_final_renamed.copy()
df_final_renamed.loc[(df_final_renamed["symbol"] == "APO") & (df_final_renamed["calendar_year"] == 2022), "revenue"] = 10968000000

In [64]:
df_final_renamed[df_final_renamed["symbol"] == "APO"]

,calendar_year,symbol,revenue,gross_profit,operating_income,net_income,price,beta,volAvg,mktCap,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
41,2019,APO,2931849000,1670762000,1.312395e+09,1536843000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
516,2020,APO,2354019000,1265321000,1.412668e+09,466802000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
991,2021,APO,5951598000,2458218000,1.805842e+09,4267895000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
1466,2022,APO,10968000000,1608167000,1.158960e+08,1994320000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False
1941,2023,APO,33213000000,32297000000,8.317000e+09,5047000000,116.55,1.615,2121695,66317416200,...,10019,5.13515,111.414854,https://financialmodelingprep.com/image-stock/...,2011-03-30,False,False,True,False,False


In [65]:
df_final_renamed[['calendar_year', 'symbol', 'name', 'sector', 'subSector', 'revenue', 'gross_profit',
       'operating_income', 'net_income', 'cik',]].to_csv("fortune_475_companies.csv", index=False)

In [66]:
df_final_renamed.groupby(["calendar_year"])["revenue"].sum()

calendar_year
2019    13093190664000
2020    12813339376000
2021    15207172285000
2022    16959515579000
2023    17391206425999
Name: revenue, dtype: int64